# Getting Started
Install dependencies and import necessary libraries.

In [ ]:
! pip install fourier-grx-dds

In [ ]:
import argparse
from fourier_grx_dds.utils import GR1ControlGroup
from fourier_grx_dds.controller import RobotController
import time

## Start the DDS bridge
DDS bridge serves as a communication bridge between motors and controller. To control the robot, you need to follow these steps:
    1. Place the robot in an appropriate position, especially the arms, and switch on the E-stop
    2. Start the DDS bridge on the machine that controls the robot by running the following cammand:

In [ ]:
! BRIDGE_CONFIG_FILE=configs/dds/config.gr1.json fftai_dds_bridge <domain_id>

Replace `configs/dds.config.gr1.json` with the path to your configuration file for DDS bridge. Default configuration files are provided under 'config/dds' directory. The configuration file defines the communication topics and IP addresses of motors. 

The `domain_id` is an identifier used by the bridge to distinguish different robots on the same network. Ensure that `domain_id` is a int variable within the range [0, 232] and unique for each robot. The `domain_id` in the configuration file must match the one specified here.

# Calibration
Before start controlling a new robot, you have to firstly calibrate the motor encoders by running the following command. Please note that, you only need to do the calibration once, as long as the calibration results are saved:

In [ ]:
! python examples/calibrate --config configs/gr1_whole_body.yaml

Replace `configs/encoders_state.yaml` with the path to your own configuration file. Replace the `encoders_state_path` with the one of your own. The calibration process will save the motor calibration values to this file.

# Start the controller
After successfully calibrating the robot, you can start the controller.

In [ ]:
config_path = "configs/gr1_upper_body.yaml"
controller = RobotController(config_path)

Replace config_path with the path to your config file for controller. We provide three example configuration files for controlling upper body, lower body and whole body, respectively. To customize configuration file:
    1. Make sure `domain_id` is consistent with the one specified when starting the bridge.
    2. Change `enable` of joints to enable or disable different joints.
    2. Replace `urdf_path` and `urdf_package_dirs` with the paths to the urdf file of the robot and its parent folder.

To enable the motors:

In [ ]:
controller.enable()

To control the robot given joint positions:

In [ ]:
# Move both arms to a specific position
left_arm_position = [
                        -0.10834163755741072, -0.07329939774949822, 0.06528929994794762, 
                        -1.4866168673727456, -0.15687147335078633, -0.13071683482883256, 
                        -0.17893611111972085  
                    ] 
right_arm_position = [
                        0.10834163755741072, -0.07329939774949822, 0.06528929994794762, 
                        -1.4866168673727456, -0.15687147335078633, -0.13071683482883256, 
                        0.17893611111972085  
                    ]
arm_position = left_arm_position + right_arm_position
controller.move_joints(GR1ControlGroup.UPPER, arm_position, duration=2.0)  

To calculate forward kinematics:

In [ ]:
# Perform forward kinematics and get the SE3 representation of the end effectors
res = controller.forward_kinematics(chain_names=["left_arm", "right_arm"])
print("SE3 of left ee:", res[0])
print("SE3 of right ee:", res[1])

To calculate inverse kinematics and move the joints to the target positions:

In [ ]:
 # Perform inverse kinematics and move the arms to the calculated position
controller.move_joints(GR1ControlGroup.UPPER, [0.0]*14, duration=2.0)
controller.inverse_kinematics(["left_arm", "right_arm"], res, move=True, velocity_scaling_factor=0.1)

To set gains of motors:

In [ ]:
# Set the gains of the right elbow pitch joint to 0
# It is now supposed to be a free joint
controller.set_gains([0], [0], [0], joint_names=["right_elbow_pitch_joint"])

To disable motors:

In [ ]:
controller.disable()

To end the control process:

In [ ]:
controller.end()